In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import numpy
import pandas

Using TensorFlow backend.


In [2]:
from sklearn.metrics import classification_report, confusion_matrix

#### Prepare data

In [94]:
trainmatchups = pandas.read_csv('kerasfiles/train.matchups.csv')
print(trainmatchups.shape)
traindata = trainmatchups.values

(25844, 69)


In [95]:
traindata = trainmatchups.values

trainX = traindata[:,1:]
trainY = traindata[:,0]

print(trainX.shape)
print(trainY.shape)

input_shape = trainX.shape[1]

(25844, 68)
(25844,)


#### Create Model

In [96]:
model = Sequential()
model.add(Dropout(0.2, input_shape=(input_shape,)))
model.add(Dense(256, kernel_initializer='normal', activation='relu'))
model.add(Dense(128, kernel_initializer='normal', activation='relu'))
model.add(Dense(64, kernel_initializer='normal', activation='relu'))
model.add(Dense(16, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

In [97]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Train Model

In [100]:
#model.fit(trainX, trainY, validation_split=0.2, epochs=5, batch_size=500, verbose=1)
model.fit(trainX, trainY, epochs=50, batch_size=500, verbose=2)

Epoch 1/50
 - 1s - loss: 0.6233 - acc: 0.6469
Epoch 2/50
 - 1s - loss: 0.6163 - acc: 0.6559
Epoch 3/50
 - 1s - loss: 0.6160 - acc: 0.6521
Epoch 4/50
 - 1s - loss: 0.6180 - acc: 0.6530
Epoch 5/50
 - 1s - loss: 0.6176 - acc: 0.6506
Epoch 6/50
 - 1s - loss: 0.6098 - acc: 0.6637
Epoch 7/50
 - 1s - loss: 0.6126 - acc: 0.6582
Epoch 8/50
 - 1s - loss: 0.6130 - acc: 0.6590
Epoch 9/50
 - 1s - loss: 0.6119 - acc: 0.6604
Epoch 10/50
 - 1s - loss: 0.6112 - acc: 0.6605
Epoch 11/50
 - 1s - loss: 0.6110 - acc: 0.6622
Epoch 12/50
 - 1s - loss: 0.6114 - acc: 0.6582
Epoch 13/50
 - 1s - loss: 0.6153 - acc: 0.6554
Epoch 14/50
 - 1s - loss: 0.6088 - acc: 0.6629
Epoch 15/50
 - 1s - loss: 0.6090 - acc: 0.6629
Epoch 16/50
 - 1s - loss: 0.6105 - acc: 0.6572
Epoch 17/50
 - 1s - loss: 0.6112 - acc: 0.6597
Epoch 18/50
 - 1s - loss: 0.6080 - acc: 0.6644
Epoch 19/50
 - 1s - loss: 0.6094 - acc: 0.6596
Epoch 20/50
 - 1s - loss: 0.6093 - acc: 0.6630
Epoch 21/50
 - 1s - loss: 0.6065 - acc: 0.6665
Epoch 22/50
 - 1s - lo

In [101]:
scores = model.evaluate(trainX, trainY)
print("\n%s: %.2f" % (model.metrics_names[0], scores[0]))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

25844/25844 [==============================] - 2s 66us/step

loss: 0.58

acc: 68.53%


#### Predict test dataset

In [102]:
testmatchups = pandas.read_csv('kerasfiles/test.matchups.csv')
print(testmatchups.shape)

(11390, 71)


In [103]:
testdata = testmatchups.values
testdata.shape

(11390, 71)

In [104]:
testdata = testmatchups.values

testID = testdata[:,0]                # ID for submission
testT = testdata[:,1].astype(int)     # played matchups during tourneys
testY = testdata[:,2].astype(int)     # truth (0 or 1) if played, 0 otherwise 
testX = testdata[:,3:]                # features

print(testID.shape)
print(testT.shape)
print(testX.shape)
print(testY.shape)

(11390,)
(11390,)
(11390, 68)
(11390,)


In [105]:
# predict all matchups
Yhat = model.predict_classes(testX)
Yprob = model.predict_proba(testX)

In [106]:
# evaluate only played matchups 
scores = model.evaluate(testX[testT==1,:], testY[testT==1])
print("\n%s: %.2f" % (model.metrics_names[0], scores[0]))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

335/335 [==============================] - 0s 131us/step

loss: 0.62

acc: 65.07%


In [107]:
# confusion matrix for played matchups
matrix = confusion_matrix(testY[testT==1], Yhat[testT==1])
matrix

array([[ 82,  91],
       [ 26, 136]])

#### Create submission file

In [89]:
pandas.DataFrame({'ID':testID,'Pred':Yprob.reshape((11390,))}).to_csv('predictions/Pred_NN_Dense.csv',index=False)